In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# Data Loading and Visualization
def load_image(image_path, size):
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])
    image = Image.open(image_path).convert('RGB')  # Ensure 3 channels even for grayscale
    return transform(image)

def load_images(image_paths, size, mask=False, trim=None):
    if trim is not None:
        image_paths = image_paths[:trim]

    images = []
    for image_path in image_paths:
        img = load_image(image_path, size)
        if mask:
            img = img[:1, :, :]  # Take only the first channel for masks
        images.append(img)

    return torch.stack(images)

def show_image(image, title=None, cmap=None, alpha=1):
    # Convert tensor to numpy and adjust channels if necessary
    image = image.permute(1, 2, 0).numpy()  
    if image.shape[-1] == 1:
        image = image[:, :, 0]  # Grayscale
    plt.imshow(image, cmap=cmap, alpha=alpha)
    if title is not None:
        plt.title(title)
    plt.axis('off')

def show_mask(image, mask, cmap=None, alpha=0.4):
    plt.imshow(image.permute(1, 2, 0).numpy())
    plt.imshow(mask.squeeze().numpy(), cmap=cmap, alpha=alpha)
    plt.axis('off')

# Data Preparation
SIZE = 256
root_path = 'C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET/Dataset_BUSI_with_GT/'
classes = sorted(os.listdir(root_path))

single_mask_paths = sorted([sorted(glob(root_path + name + "/*mask.png")) for name in classes])
double_mask_paths = sorted([sorted(glob(root_path + name + "/*mask_1.png")) for name in classes])

image_paths = []
mask_paths = []
for class_path in single_mask_paths:
    for path in class_path:
        img_path = path.replace('_mask','')
        image_paths.append(img_path)
        mask_paths.append(path)

# Load images and masks
images = load_images(image_paths, SIZE)
masks = load_images(mask_paths, SIZE, mask=True)

# ... (rest of your data visualization and preprocessing code)

# Model Definition
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.1, pooling=True):
        super(EncoderBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.dropout = nn.Dropout2d(dropout_rate)
        self.pool = nn.MaxPool2d(2) if pooling else nn.Identity()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout(x)
        pooled_x = self.pool(x)
        return pooled_x, x

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.1):
        super(DecoderBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_block = EncoderBlock(in_channels, out_channels, dropout_rate, pooling=False)

    def forward(self, x):
        x, skip_connection = x  # Unpack the tuple
        x = self.up(x)
        x = torch.cat([x, skip_connection], dim=1) 
        x = self.conv_block(x)  # Pass the concatenated tensor
        return x[1]  # Return only the second tensor from the tuple

class AttentionGate(nn.Module):
    def __init__(self, in_channels, gating_channels, inter_channels=None, use_batchnorm=True):
        super(AttentionGate, self).__init__()
        self.use_batchnorm = use_batchnorm

        if inter_channels is None:
            inter_channels = in_channels // 2

        self.theta_conv = nn.Conv2d(in_channels, inter_channels, kernel_size=2, stride=2, bias=False)
        self.phi_conv = nn.Conv2d(gating_channels, inter_channels, kernel_size=1, bias=False)
        self.psi_conv = nn.Conv2d(inter_channels, 1, kernel_size=1, bias=False)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.batchnorm = nn.BatchNorm2d(gating_channels)

    def forward(self, x):
        # Encoder
        enc1, skip1 = self.encoder1(x)
        enc2, skip2 = self.encoder2(enc1)
        enc3, skip3 = self.encoder3(enc2)
        enc4, skip4 = self.encoder4(enc3)

        # Bottleneck
        bottleneck = self.bottleneck(enc4)

        # Decoder with Attention Gates
        # Extract the tensor from the skip connection tuple
        attn1 = self.attn1(bottleneck, skip4[1])  # Use skip4[1] instead of skip4
        dec1 = self.decoder1(bottleneck, attn1)
        attn2 = self.attn2(dec1, skip3[1])
        dec2 = self.decoder2(dec1, attn2)
        attn3 = self.attn3(dec2, skip2[1])
        dec3 = self.decoder3(dec2, attn3)
        attn4 = self.attn4(dec3, skip1[1])
        dec4 = self.decoder4(dec3, attn4)

        # Output
        output = self.output_conv(dec4)
        output = self.sigmoid(output)
        return output
        
# U-Net with Attention Gates
class AttentionUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(AttentionUNet, self).__init__()

        # Encoder
        self.encoder1 = EncoderBlock(in_channels, 32)
        self.encoder2 = EncoderBlock(32, 64)
        self.encoder3 = EncoderBlock(64, 128)
        self.encoder4 = EncoderBlock(128, 256)

        # Bottleneck
        self.bottleneck = EncoderBlock(256, 512, pooling=False)

        # Decoder with Attention Gates
        self.attn1 = AttentionGate(512, 256)
        self.decoder1 = DecoderBlock(512, 256)
        self.attn2 = AttentionGate(256, 128)
        self.decoder2 = DecoderBlock(256, 128)
        self.attn3 = AttentionGate(128, 64)
        self.decoder3 = DecoderBlock(128, 64)
        self.attn4 = AttentionGate(64, 32)
        self.decoder4 = DecoderBlock(64, 32)

        # Output
        self.output_conv = nn.Conv2d(32, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Encoder
        enc1, skip1 = self.encoder1(x)
        enc2, skip2 = self.encoder2(enc1)
        enc3, skip3 = self.encoder3(enc2)
        enc4, skip4 = self.encoder4(enc3)

        # Bottleneck
        bottleneck = self.bottleneck(enc4)

        # Decoder with Attention Gates
        attn1 = self.attn1(bottleneck, skip4[1])  
        dec1 = self.decoder1((bottleneck, attn1))  # Pass as a tuple
        attn2 = self.attn2(dec1, skip3[1])
        dec2 = self.decoder2((dec1, attn2))
        attn3 = self.attn3(dec2, skip2[1])
        dec3 = self.decoder3((dec2, attn3)) 
        attn4 = self.attn4(dec3, skip1[1])
        dec4 = self.decoder4((dec3, attn4))

        # Output
        output = self.output_conv(dec4)
        output = self.sigmoid(output)
        return output

# Model Instantiation and Training
model = AttentionUNet()

# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
# Define num_epochs
num_epochs = 50
batch_size = 8

# Training Loop
for epoch in range(num_epochs): 
    for i in range(0, len(images), batch_size):
        # Get a batch of images and masks
        image_batch = images[i:i+batch_size]
        mask_batch = masks[i:i+batch_size]

        optimizer.zero_grad()
        output = model(image_batch)
        loss = criterion(output, mask_batch)
        loss.backward()
        optimizer.step()

        # ... (Add validation and logging as needed)



TypeError: AttentionGate.forward() takes 2 positional arguments but 3 were given

In [ ]:
        # ... (Add validation and logging as needed)

# Model Saving
torch.save(model.state_dict(), 'model-regularization.pth')

# Model Loading
model = AttentionUNet()
model.load_state_dict(torch.load('model-regularization.pth'))
model.eval()  # Set to evaluation mode

# ... (Rest of your evaluation and visualization code)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Data Loading Functions
def load_image(image_path, SIZE):
    try:
        image = load_img(image_path)  # Load image
        image = img_to_array(image)   # Convert to numpy array
        image = image / 255.0         # Normalize the image
        image = tfi.resize(image, (SIZE, SIZE))  # Resize image
        return np.round(image, 4)     # Round to 4 decimal places
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None  # Handle error

def load_images(image_paths, SIZE, mask=False):
    images = []
    for image_path in image_paths:
        image = load_image(image_path, SIZE)
        if image is not None:
            if mask:
                image = image[:, :, :1]  # Take only 1 channel for masks
            images.append(image)
    return np.array(images)

def show_image(image, title=None, cmap=None, alpha=1):
    plt.imshow(image, cmap=cmap, alpha=alpha)
    if title:
        plt.title(title)
    plt.axis('off')

def show_mask(image, mask, cmap=None, alpha=0.4):
    plt.imshow(image)
    plt.imshow(mask.squeeze(), cmap=cmap, alpha=alpha)
    plt.axis('off')

SIZE = 256
root_path = 'C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET/Dataset_BUSI_with_GT/'

# Collect image and mask paths
classes = sorted(glob(root_path + '*'))

import os

if not os.path.exists(root_path):
    print(f"Directory {root_path} does not exist.")
else:
    print(f"Directory {root_path} exists.")

classes = sorted(os.listdir(root_path))
print(f"Classes (Subdirectories): {classes}")

for class_name in classes:
    print(f"Contents of {class_name} folder: {os.listdir(os.path.join(root_path, class_name))[:5]}")

single_mask_paths = sorted([sorted(glob(root_path + name + "/*mask.png")) for name in classes])
double_mask_paths = sorted([sorted(glob(root_path + name + "/*mask_1.png")) for name in classes])

image_paths = []
mask_paths = []
for class_path in single_mask_paths:
    for path in class_path:
        img_path = path.replace('_mask', '')
        image_paths.append(img_path)
        mask_paths.append(path)

images = load_images(image_paths, SIZE)
masks = load_images(mask_paths, SIZE, mask=True)

print(f"Image Paths: {image_paths[:5]}")  # Print first 5 image paths
print(f"Mask Paths: {mask_paths[:5]}")    # Print first 5 mask paths

# PyTorch Dataset and DataLoader

class BreastCancerDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        return image, mask


# Define Transform
transform = transforms.Compose([
    transforms.ToTensor()
])

# PyTorch Dataset and DataLoader
dataset = BreastCancerDataset(images, masks, transform=transform)
print(f"Number of samples in dataset: {len(dataset)}")

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# UNet Architecture in PyTorch

class EncoderBlock(nn.Module):
    def __init__(self, filters, rate, pooling=True):
        super(EncoderBlock, self).__init__()
        self.filters = filters
        self.rate = rate
        self.pooling = pooling
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=filters, out_channels=filters, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(rate)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.dropout(x)
        x = torch.relu(self.conv2(x))
        if self.pooling:
            return self.pool(x), x
        else:
            return x

class DecoderBlock(nn.Module):
    def __init__(self, filters, rate):
        super(DecoderBlock, self).__init__()
        self.filters = filters
        self.rate = rate
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv1 = nn.Conv2d(in_channels=filters, out_channels=filters, kernel_size=3, padding=1)

    def forward(self, x, skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)  # Concatenate
        x = torch.relu(self.conv1(x))
        return x

class AttentionGate(nn.Module):
    def __init__(self, filters):
        super(AttentionGate, self).__init__()
        self.conv1 = nn.Conv2d(filters, filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, stride=2, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, skip):
        x1 = torch.relu(self.conv1(x))
        x2 = torch.relu(self.conv2(skip))
        x = torch.add(x1, x2)
        return torch.mul(self.sigmoid(x), skip)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder1 = EncoderBlock(32, 0.1)
        self.encoder2 = EncoderBlock(64, 0.1)
        self.encoder3 = EncoderBlock(128, 0.2)
        self.encoder4 = EncoderBlock(256, 0.2)
        
        self.center = EncoderBlock(512, 0.3, pooling=False)
        
        self.attention1 = AttentionGate(256)
        self.decoder1 = DecoderBlock(256, 0.2)
        
        self.attention2 = AttentionGate(128)
        self.decoder2 = DecoderBlock(128, 0.2)
        
        self.attention3 = AttentionGate(64)
        self.decoder3 = DecoderBlock(64, 0.1)
        
        self.attention4 = AttentionGate(32)
        self.decoder4 = DecoderBlock(32, 0.1)
        
        self.final_conv = nn.Conv2d(32, 1, kernel_size=1)
    
    def forward(self, x):
        p1, c1 = self.encoder1(x)
        p2, c2 = self.encoder2(p1)
        p3, c3 = self.encoder3(p2)
        p4, c4 = self.encoder4(p3)
        
        center = self.center(p4)
        
        a1 = self.attention1(center, c4)
        d1 = self.decoder1(center, a1)
        
        a2 = self.attention2(d1, c3)
        d2 = self.decoder2(d1, a2)
        
        a3 = self.attention3(d2, c2)
        d3 = self.decoder3(d2, a3)
        
        a4 = self.attention4(d3, c1)
        d4 = self.decoder4(d3, a4)
        
        return torch.sigmoid(self.final_conv(d4))

# Initialize Model
model = UNet()

# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(20):
    model.train()
    running_loss = 0.0
    for images, masks in dataloader:
        images = images.permute(0, 3, 1, 2)  # Reshape for PyTorch (B, C, H, W)
        masks = masks.permute(0, 3, 1, 2)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

# Example Visualization of Prediction (after training)
image, mask = next(iter(dataloader))
model.eval()
with torch.no_grad():
    output = model(image.permute(0, 3, 1, 2))

output = output.squeeze(1)  # Remove channel dimension
plt.imshow(output[0].cpu().numpy(), cmap='gray')
plt.show()


Directory C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET/Dataset_BUSI_with_GT/ exists.
Classes (Subdirectories): ['benign', 'malignant', 'normal']
Contents of benign folder: ['benign (1).png', 'benign (1)_mask.png', 'benign (10).png', 'benign (10)_mask.png', 'benign (100).png']
Contents of malignant folder: ['malignant (1).png', 'malignant (1)_mask.png', 'malignant (10).png', 'malignant (10)_mask.png', 'malignant (100).png']
Contents of normal folder: ['normal (1).png', 'normal (1)_mask.png', 'normal (10).png', 'normal (10)_mask.png', 'normal (100).png']
Error loading image C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET/Dataset_BUSI_with_GT/benign\benign (1).png: name 'load_img' is not defined
Error loading image C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET/Dataset_BUSI_with_GT/benign\benign (10).png: name 'load_img' is not defined
Error loading image C://Users/Muruganantham J/Documents/Research/Codes/BreastCancer_UNET

ValueError: num_samples should be a positive integer value, but got num_samples=0